In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../Dell/Downloads/WELFake_Dataset.csv')
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [3]:
df=df.dropna()

In [4]:
X=df.drop(['label','Unnamed: 0'],axis=1)
y=df['label']

In [5]:
X.shape

(71537, 2)

In [6]:
import tensorflow as tf

In [7]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [8]:
voc_size=5000

In [9]:
message=X.copy()
message.reset_index(inplace=True)

In [10]:
import nltk
import re
from nltk.corpus import stopwords

In [11]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(message)):
    review=re.sub('[^a-zA-Z]',' ',message['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [12]:
onehot_rep=[one_hot(words,voc_size) for words in corpus]
onehot_rep

[[4887, 4981, 2310, 831, 1261, 4822, 627, 4627, 3473, 1649, 4734, 1320],
 [625,
  3041,
  1602,
  3703,
  3516,
  1517,
  1468,
  1516,
  500,
  717,
  936,
  844,
  2619,
  1320],
 [2930, 4510, 1588, 468, 3442, 1312, 3100, 3622, 4951, 1740, 1137, 3066],
 [181, 1481, 2607, 80, 3862, 4846, 2629, 2126, 497, 2810, 1702],
 [1527, 3100, 4605, 1359, 4495, 2625, 1621, 4722, 4605],
 [1537, 4765, 1480, 4938, 702, 4148, 3518, 1431, 2698, 3020, 853],
 [2912, 999, 605, 1610, 1481, 4133, 1312, 241, 2127, 1320],
 [248,
  1256,
  1875,
  928,
  4603,
  4186,
  1089,
  559,
  3884,
  248,
  1427,
  3638,
  4926,
  4103,
  1320],
 [1799, 4369, 1555, 4293, 3153, 1812, 4260, 4369],
 [200, 904, 1120, 3962, 2302, 2812, 367, 1399],
 [2296, 1454, 2786, 3139, 4039, 4159, 3473, 3312, 3473, 167],
 [1629, 1403, 1610, 4212, 4914, 1426, 4040, 3449, 3023],
 [4038, 439, 1510, 1403, 4647, 377, 2901, 2813, 4477, 607, 2495],
 [2081, 3116, 6, 151, 2882, 1515, 2046, 463, 2043, 4619],
 [248, 90, 4949, 4641, 3271, 2912, 37

In [13]:
sent_length=20
embedding_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedding_docs)

[[   0    0    0 ... 1649 4734 1320]
 [   0    0    0 ...  844 2619 1320]
 [   0    0    0 ... 1740 1137 3066]
 ...
 [   0    0    0 ... 4323 2874 3835]
 [   0    0    0 ... 2389 3157 4225]
 [   0    0    0 ... 2851 2813  125]]


In [14]:
len(embedding_docs)

71537

In [15]:
embedding_vectorfeature=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vectorfeature,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
X_final=np.array(embedding_docs)
y_final=np.array(y)

In [17]:
X_final.shape,y_final.shape

((71537, 20), (71537,))

In [21]:
from  sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X_final, y_final, test_size=0.33, random_state=42)

In [22]:
model.fit(X_final,y_final,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
1118/1118 [==============================] - 41s 35ms/step - loss: 0.2960 - accuracy: 0.8688 - val_loss: 0.2104 - val_accuracy: 0.9146
Epoch 2/10
1118/1118 [==============================] - 36s 33ms/step - loss: 0.2143 - accuracy: 0.9120 - val_loss: 0.1809 - val_accuracy: 0.9356
Epoch 3/10
1118/1118 [==============================] - 36s 33ms/step - loss: 0.1837 - accuracy: 0.9253 - val_loss: 0.1427 - val_accuracy: 0.9478
Epoch 4/10
1118/1118 [==============================] - 38s 34ms/step - loss: 0.1592 - accuracy: 0.9357 - val_loss: 0.1165 - val_accuracy: 0.9554
Epoch 5/10
1118/1118 [==============================] - 37s 33ms/step - loss: 0.1344 - accuracy: 0.9458 - val_loss: 0.1070 - val_accuracy: 0.9581
Epoch 6/10
1118/1118 [==============================] - 37s 33ms/step - loss: 0.1137 - accuracy: 0.9536 - val_loss: 0.0824 - val_accuracy: 0.9683
Epoch 7/10
1118/1118 [==============================] - 32s 29ms/step - loss: 0.0956 - accuracy: 0.9614 - val_loss: 0.0729 -

In [27]:
y_pred=model.predict(X_test).round()

738/738 [==============================] - 3s 4ms/step


In [30]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [29]:
confusion_matrix(y_test,y_pred)

array([[11573,   120],
       [  211, 11704]], dtype=int64)

In [32]:
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     11693
           1       0.99      0.98      0.99     11915

    accuracy                           0.99     23608
   macro avg       0.99      0.99      0.99     23608
weighted avg       0.99      0.99      0.99     23608

0.985979329041003


In [38]:
def cleaning(data):
    review=re.sub('[^a-zA-Z]',' ',data)
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    return review

In [43]:
data=pd.DataFrame({"text":[df.loc[0,'title']]})

,text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...


In [51]:
output=cleaning('LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]')

In [52]:
output

'law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video'

In [53]:
onehot_output=[one_hot(output,voc_size) ]

In [54]:
onehot_output

[[4887, 4981, 2310, 831, 1261, 4822, 627, 4627, 3473, 1649, 4734, 1320]]

In [56]:
embed_output=pad_sequences(onehot_output,maxlen=20,padding='pre')
embed_output

array([[   0,    0,    0,    0,    0,    0,    0,    0, 4887, 4981, 2310,
         831, 1261, 4822,  627, 4627, 3473, 1649, 4734, 1320]])

In [65]:
model.predict(embed_output).round()[0][0]

1/1 [==============================] - 0s 24ms/step


1.0

In [72]:
def manual_testing(data):
    review=re.sub('[^a-zA-Z]',' ',data)
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    onehot_output=[one_hot(review,voc_size)]
    embed_output=pad_sequences(onehot_output,maxlen=20,padding='pre')
    return model.predict(embed_output).round()[0][0]

In [73]:
news=str(input())
manual_testing(news)

OBAMACARE: Wheels Slowly Falling Off The Socialized Healthcare Trainâ€¦Billions Wasted
1/1 [==============================] - 0s 16ms/step


1.0